In [1]:
import tensorflow as tf
print(tf.__version__)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

1.14.0


In [ ]:
!conda upgrade tensorflow


In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json 
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file =open('C:\\Users\\amsarang\\Documents\\chatbot\\intents.json').read()
intents  = json.loads(data_file)

Using TensorFlow backend.


In [10]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w,intent['tag']))
        
        #add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [17]:
#Lemmatize,lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#Sort Classes
classes = sorted(list(set(classes)))

#Documents = combination between patterns and intents
print(len(documents), "documents")

#classes = intents
print(len(classes), "classes", classes)
print("\n")

#Words = all words, vocabulary
print(len(words), "Unique Lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))


141 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


88 Unique Lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [25]:
#Creating our training Data
training = []

#Create an empty array for our output
output_empty = [0] * len(classes)

#training set, bag of words for each sentence
for doc in documents:
    #Initialize our bag of words
    bag = []
    
    #List of tokenized words for the pattern
    pattern_words = doc[0]
    
    #Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    #Create our bag of words array with l, if word found match in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    #Output is a '0' for each tag and '1' for each current tag(for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
        
    training.append([bag, output_row])
        
#Shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
    
#Create train and test lists. X - Patterns, Y - Intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training Data Created")

Training Data Created


In [30]:
#Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 
#3rd output layer contains number of neurons

#Equal to number of intents to predict output intent with softmax

model = Sequential()
#model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(train_y[0]), activation = 'softmax'))

#Compile model. Stochastic gradient descent with Nesterov accelerated gradient
#gives good results for the model
sgd = SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
141/141 [==============================] - 1s 5ms/step - loss: 2.2243 - accuracy: 0.0638
Epoch 2/200
141/141 [==============================] - 0s 359us/step - loss: 2.0469 - accuracy: 0.2695
Epoch 3/200
141/141 [==============================] - 0s 374us/step - loss: 1.8137 - accuracy: 0.4326
Epoch 4/200
141/141 [==============================] - 0s 362us/step - loss: 1.4971 - accuracy: 0.5532
Epoch 5/200
141/141 [==============================] - 0s 445us/step - loss: 1.2151 - accuracy: 0.6950
Epoch 6/200
141/141 [==============================] - 0s 432us/step - loss: 0.8469 - accuracy: 0.7660
Epoch 7/200
141/141 [==============================] - 0s 341us/step - loss: 0.6198 - accuracy: 0.8156
Epoch 8/200
141/141 [==============================] - 0s 442us/step - loss: 0.6319 - accuracy: 0.8156
Epoch 9/200
141/141 [==============================] - 0s 352us/step - loss: 0.4083 - accuracy: 0.9220
Epoch 10/200
141/141 [==============================] - 0s 363us/step - los

141/141 [==============================] - 0s 344us/step - loss: 0.0065 - accuracy: 1.0000
Epoch 158/200
141/141 [==============================] - 0s 362us/step - loss: 0.0051 - accuracy: 1.0000
Epoch 159/200
141/141 [==============================] - 0s 481us/step - loss: 0.0163 - accuracy: 0.9929
Epoch 160/200
141/141 [==============================] - 0s 472us/step - loss: 0.0059 - accuracy: 1.00000s - loss: 0.0036 - accuracy: 1.00
Epoch 161/200
141/141 [==============================] - 0s 454us/step - loss: 0.0127 - accuracy: 0.9929
Epoch 162/200
141/141 [==============================] - 0s 459us/step - loss: 0.0088 - accuracy: 0.9929
Epoch 163/200
141/141 [==============================] - 0s 500us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 164/200
141/141 [==============================] - 0s 511us/step - loss: 0.0077 - accuracy: 1.0000
Epoch 165/200
141/141 [==============================] - 0s 428us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 166/200
141/141 [============

In [32]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

intents = json.loads(open('C:\\Users\\amsarang\\Documents\\chatbot\\intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [33]:
def clean_up_sentence(sentence):
    #tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    #stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
#return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    #Tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    #bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                #assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in Bag: %s", w)
    return (np.array(bag))

def predict_class(sentence, model):
    #Filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results  = [[i,r] for i,r in enumeration(res) if r>ERROR_THRESHOLD]
    
    #sort by strength of probability
    results.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    
    for r in results:
        return_list.append({"Intent ": classes[r[0]], "Probability ": str(r[1])})
    return return_list

In [34]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']==tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [36]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-36-2bc521bd8c5e>", line 15, in send
    res = chatbot_response(msg)
  File "<ipython-input-34-01141d200ac0>", line 11, in chatbot_response
    ints = predict_class(text, model)
  File "<ipython-input-33-4910a4f6e90d>", line 30, in predict_class
    results  = [[i,r] for i,r in enumeration(res) if r>ERROR_THRESHOLD]
NameError: name 'enumeration' is not defined


In [37]:
pwd

'C:\\Users\\amsarang'

In [ ]:
#Tensorflow is supported by Python 64 Bit only